In [8]:
import math
import torch
import torch.nn as nn
import torch.optim as optim

Refer to https://www.datacamp.com/tutorial/building-a-transformer-with-py-torch

In [9]:
class PositionWiseFFN(nn.Module):
    """Same MLP applied to all token(position) representations"""
    def __init__(self, emb_dim, ffn_dim):
        super().__init__()
        self.fc1 = nn.Linear(emb_dim, ffn_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ffn_dim, emb_dim)
    
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_dim, max_seq_len):
        super().__init__()

        pe = torch.zeros(max_seq_len, emb_dim)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, emb_dim, 2).float() * -(math.log(10000.0) / emb_dim))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        assert emb_dim % num_heads == 0, "Embedding dimension must be divided by number of heads"

        # Dimensions initialization
        self.emb_dim = emb_dim
        self.num_heads = num_heads
        # all features are divided into multi head, each head have a part of features
        self.head_emb_dim = self.emb_dim // self.num_heads

        # Transformation matrixs
        self.W_q = nn.Linear(emb_dim, emb_dim)
        self.W_k = nn.Linear(emb_dim, emb_dim)
        self.W_v = nn.Linear(emb_dim, emb_dim)
        self.W_o = nn.Linear(emb_dim, emb_dim)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_emb_dim)

        # Mask scores (where positions are 0) with near negative inf
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Apply sofxmax to attention scores
        attn_scores = torch.softmax(attn_scores, dim=-1)

        # Get the final output
        output = torch.matmul(attn_scores, V)
        return output
    
    def split(self, x):
        # Reshape the input emb_dim (to multi-head, each head owns a part of input features) for multi-head attention
        batch_size, seq_len, emb_dim = x.size()
        # transpose to fix batch_size and num_heads, let seq_len, head_emb_dim participate in matrix multiplication
        return x.view(batch_size, seq_len, self.num_heads, self.head_emb_dim).transpose(1, 2)

    def combine(self, x):
        batch_size, num_heads, seq_len, head_emb_dim = x.size()
        # contiguous() ensures the memory layout of the tensor is contiguous
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.emb_dim)
    
    def forward(self, Q, K, V, mask=None):
        # Split input to multi heads
        Q = self.split(self.W_q(Q))
        K = self.split(self.W_k(K))
        V = self.split(self.W_v(V))

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # Combine outputs and apply transformation
        output = self.W_o(self.combine(attn_output))
        return output


In [12]:
class EncoderLayer(nn.Module):
    def __init__(self, emb_dim, num_heads, ffn_dim, dropout):
        super().__init__()
        self.self_atten = MultiHeadAttention(emb_dim, num_heads)
        self.ffn = PositionWiseFFN(emb_dim, ffn_dim)
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_atten(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x

In [13]:
class DecoderLayer(nn.Module):
    def __init__(self, emb_dim, num_heads, ffn_dim, dropout):
        super().__init__()
        self.self_attn = MultiHeadAttention(emb_dim, num_heads)
        self.cross_attn = MultiHeadAttention(emb_dim, num_heads)
        self.ffn = PositionWiseFFN(emb_dim, ffn_dim)
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)
        self.norm3 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        # Masked Multiple Head Attention
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ffn_output = self.ffn(x)
        x = self.norm3(x + self.dropout(ffn_output))
        return x

In [14]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, emb_dim, num_heads,
                num_layers, ffn_dim, max_seq_len, dropout):
        super().__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, emb_dim)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, emb_dim)
        self.positional_encoding = PositionalEncoding(emb_dim, max_seq_len)

        self.encoder_layers = nn.ModuleList([EncoderLayer(emb_dim, num_heads, ffn_dim, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(emb_dim, num_heads, ffn_dim, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(emb_dim, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2) # 0s stand for padding tokens
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3) # unsqueeze(1) add new dimension at axis 1
        seq_len = tgt.size(1)
        nopeak_mask = (1-torch.triu(torch.ones(1, seq_len, seq_len), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.gernerate_mask(src, tgt)
        # inputEmb + PosEmb is implemented in self.positional_encoding())
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)
        
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
        
        output = self.fc(dec_output)
        return output

In [15]:
src_vocab_size = 50
tgt_vocab_size = 50
emb_dim = 18
num_heads = 3
num_layers = 2
ffn_dim = 24
max_seq_len = 6
dropout = 0.1

In [27]:
transformer = Transformer(src_vocab_size, tgt_vocab_size, emb_dim, num_heads, num_layers, ffn_dim, max_seq_len, dropout)

src_data = torch.randint(1, src_vocab_size, (8, max_seq_len))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (8, max_seq_len))  # (batch_size, seq_length)
src_data[:, 2:5] = 0

In [29]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(5):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    a = output.contiguous().view(-1, tgt_vocab_size)
    b = tgt_data[:, 1:].contiguous().view(-1)
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 4.087765216827393
Epoch: 2, Loss: 4.064812660217285
Epoch: 3, Loss: 4.055843353271484
Epoch: 4, Loss: 4.092369079589844
Epoch: 5, Loss: 4.115525245666504


In [30]:
transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (8, max_seq_len))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (8, max_seq_len))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: 4.152605056762695
